In [2]:
from datetime import datetime
import datetime as dt
import time
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import duckdb as db
import seaborn as sns
import matplotlib.pyplot as plt
import IPython as ip
import  os as os

In [11]:
daysLimit = int(input('How many days back in history?'))

In [4]:
sg = Subgrounds()
apiKey = input('API Key, please')

In [5]:
#for when we save the files
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".
file = file+'/data'
if not os.path.exists(file):
    os.makedirs(file)

*******************************************************************
*******************************************************************
LET'S LOOK AT SILO BORROWS FIRST

In [6]:
#SILO subgraph: https://gateway.thegraph.com/api/[api-key]/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB
silo = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB')

##this captures the list of people who borrowed XAI against gOHM
borrows = silo.Query.borrows(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

borrowsDb = sg.query_df([
    borrows.id
    ,borrows.hash
    ,borrows.logIndex
    ,borrows.blockNumber
    ,borrows.timestamp
    ,borrows.account
    ,borrows.market
    ,borrows.market.rates
    ,borrows.asset
    ,borrows.amount
    ,borrows.amountUSD 
])
borrowsDb.head(100)

#this, in turn, pulls repayment events
repays = silo.Query.repays(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

repaysDb = sg.query_df([
    repays.id
    ,repays.hash
    ,repays.logIndex
    ,repays.blockNumber
    ,repays.timestamp
    ,repays.account
    ,repays.market
    ,repays.market.rates
    ,repays.asset
    ,repays.amount
    ,repays.amountUSD
    ,repays.position
])
repaysDb.head(100)

In [12]:
oldSiloBorrowsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmBorrows.csv'
try:
    oldSiloBorrowsDb = pd.read_csv (siloFileName)
except:
    oldSiloBorrowsDb = pd.DataFrame()
oldSiloBorrowsDb.head(100)

""


In [13]:
maxSiloBorrowTimestamp = 0
try:
    maxSiloBorrowTimestamp = (max(0,oldSiloBorrowsDb['transaction_timestamp'].max()))-5
except:
        maxSiloBorrowTimestamp = 0
maxSiloBorrowTimestamp = int(maxSiloBorrowTimestamp)
maxSiloBorrowTimestamp

0

In [14]:
# 0xc413dd03555f3eb29d834b482d386b2999dc2eb0

In [15]:
##this captures the ENTIRE list of people who borrowed stablecoins against gOHM
borrowTicker = 0
borrowslist = pd.DataFrame()
siloBorrowsDb = pd.DataFrame()
borrowListLength = 1000
datediff = 0
now=0
daysAgo=0
exit = False
while exit==False:
    skipValue = (borrowTicker)*(1000)
    borrows = silo.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
                ,'timestamp_gte': maxSiloBorrowTimestamp
                } #gOHM market
    )

    borrowslist = sg.query_df([
        borrows.id
        ,borrows.hash
        ,borrows.blockNumber
        ,borrows.timestamp
        ,borrows.account.id
        ,borrows.market.id
        ,borrows.market.protocol.id
        ,borrows.market.rates.rate
        ,borrows.market.rates.duration
        ,borrows.market.rates.side
        ,borrows.market.rates.type
        ,borrows.market.rates.token.id
        ,borrows.asset.name
        ,borrows.asset.symbol
        ,borrows.asset.decimals
        ,borrows.asset.lastPriceUSD
        ,borrows.amount
        ,borrows.amountUSD
    ])

    siloBorrowsDb=pd.concat([siloBorrowsDb, borrowslist])
    recordTimestamp1 = siloBorrowsDb.iat[borrowTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    siloBorrowsDbLength = len(siloBorrowsDb)
    borrowListLength = len(borrowslist)
    recordID = siloBorrowsDb.iat[borrowTicker,0]
    borrow = siloBorrowsDb.iat[borrowTicker,1]
    print("iterations: ", borrowTicker, "Lines skipped: ",skipValue, "records collected: ", borrowListLength, " - latest recordID: ",borrow, " ", recordID , " - borrows DB length: ", siloBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", borrowTicker, "records collected: ", borrowListLength, " - userdb length: ", borrowsDbLength)
    borrowTicker = borrowTicker+1

    if daysAgo>daysLimit: exit =True
    if borrowListLength<1000: exit =True

siloBorrowsDb.columns = siloBorrowsDb.columns.str.replace("borrows", "transaction")
siloBorrowsDb['event']='borrow'
siloBorrowsDb.drop_duplicates
print("Done. Total Records collected: ", siloBorrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  162  - latest recordID:  0x84778306b3f662f7d98e431591f302d924d143c3c0b281d067cb506565b9f960   0x84778306b3f662f7d98e431591f302d924d143c3c0b281d067cb506565b9f960-364  - borrows DB length:  162  - latest record from:  2023-02-21 09:50:23 0 days ago
Done. Total Records collected:  162


In [16]:
siloBorrowsDb['transactionReadableAmount'] = siloBorrowsDb['transaction_amount']/(10**(siloBorrowsDb['transaction_asset_decimals']))
siloBorrowsDb['transactionReadableAmount']=siloBorrowsDb['transactionReadableAmount'].astype('float')
siloBorrowsDb['dateTime'] =(pd.to_datetime(siloBorrowsDb['transaction_timestamp'],unit='s'))

siloBorrowEventsDbTimedClean = siloBorrowsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloBorrowEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])

siloBorrowEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloBorrowEventsDbTimedClean['calDate'] = siloBorrowEventsDbTimedClean['dateTime'].dt.normalize()
siloBorrowEventsDbTimedClean.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,borrow,0x84778306b3f662f7d98e431591f302d924d143c3c0b2...,16678471,1677001823,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,6540.000000,1.002384,6588.024198,2023-02-21 17:50:23,2023-02-21
6,borrow,0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5...,16651434,1676673563,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,60500.000000,1.002384,59947.550425,2023-02-17 22:39:23,2023-02-17
12,borrow,0xd9329978828295f4f091eaa77a6d8c388c5616bbe92c...,16644592,1676590307,0x96035b45bc0d22f0ed11283a91145fb001c48370,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Wrapped Ether,WETH,0.600000,1675.554141,995.052978,2023-02-16 23:31:47,2023-02-16
18,borrow,0x3b0f26e10c63a71c916e8a68bd1889d9bab38aeb62d7...,16641746,1676556011,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,100000.000000,1.002384,100012.487283,2023-02-16 14:00:11,2023-02-16
24,borrow,0xfa292276fc71f90e29ca90a7b8a2be9a4094f459fb99...,16634870,1676472911,0x16ff8ad104fa6195260a6d4a3f311c5f120df77d,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,3000.000000,1.002384,3023.153053,2023-02-15 14:55:11,2023-02-15
30,borrow,0xd5dc7147192591c7d19856d12e9278928932a5921653...,16627480,1676383655,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,1000.000000,1.002384,992.079025,2023-02-14 14:07:35,2023-02-14
36,borrow,0x39c11cb01e34d9f8ca55a8820482eaf8a217033cdb6f...,16626801,1676375459,0x16ff8ad104fa6195260a6d4a3f311c5f120df77d,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Wrapped Ether,WETH,1.920000,1675.554141,2885.217480,2023-02-14 11:50:59,2023-02-14
42,borrow,0xf1d5b9c7f23bd9935061e4d08734cbfba9b9a53bacd5...,16626396,1676370563,0x339fd7c17ce670e7821f2f2ee94ddd49771483da,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,4000.000000,1.002384,3963.627622,2023-02-14 10:29:23,2023-02-14
48,borrow,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,16623063,1676330255,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,2457.516074,1.002384,2446.717700,2023-02-13 23:17:35,2023-02-13
54,borrow,0x3dba2dd3c913b399164efd71db73121c9c94c0096139...,16622907,1676328371,0x46535cbc3fa574c32ea392d4f89c518290597072,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,26.400000,2756.217547,71910.521424,2023-02-13 22:46:11,2023-02-13


In [17]:
path =file+'/siloFinanceGohmBorrows.csv'
siloBorrowEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmBorrows_piped.csv'
siloBorrowEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloBorrowEventsDbTimedClean),' records')

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/siloFinanceGohmBorrows_piped.csv file saved, 27  records


In [18]:
totalBorrowedUSDSilo = sum(siloBorrowEventsDbTimedClean['transaction_amountUSD'])
print('A total of $',round(totalBorrowedUSDSilo,2),  'has been borrowed in different stablecoins over time over at Silo')

A total of $ 571167.73 has been borrowed in different stablecoins over time over at Silo


In [19]:
gOHMBorrowedSilo = db.query("Select distinct "
                           "transaction_hash "
                           ",transaction_timestamp "
                           ", dateTime "
                           ",transaction_account_id "
                           ", transaction_asset_symbol "
                           ", transactionReadableAmount "
                           ", transaction_amountUSD "
                           "from siloBorrowsDb where lower(transaction_asset_symbol) = 'gohm'").df()

gOHMBorrowedSilo.head(10)

,transaction_hash,transaction_timestamp,dateTime,transaction_account_id,transaction_asset_symbol,transactionReadableAmount,transaction_amountUSD
0,0x3dba2dd3c913b399164efd71db73121c9c94c0096139...,1676328371,2023-02-13 22:46:11,0x46535cbc3fa574c32ea392d4f89c518290597072,gOHM,26.400,71910.521424
1,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,1669229867,2022-11-23 18:57:47,0x7776421dd146dfd6ea947cbf29b69c519807d259,gOHM,0.182,399.040935


In [21]:
totalSiloBorrowedgOHM = sum(gOHMBorrowedSilo['transactionReadableAmount'])
totalSiloBorrowedgOHMinUSD = sum(gOHMBorrowedSilo['transaction_amountUSD'])
print('A total of ',round(totalSiloBorrowedgOHM,2),  'OHM has been borrowed in different stablecoins over time in Silo, roughly USD$',totalSiloBorrowedgOHMinUSD)

A total of  26.58 OHM has been borrowed in different stablecoins over time in Silo, roughly USD$ 72309.56235872359


In [23]:
path =file+'/ohmBorrowedSilo.csv'
gOHMBorrowedSilo.to_csv(path, index = False)

path =file+'/ohmBorrowedSilo_piped.csv'
gOHMBorrowedSilo.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(gOHMBorrowedSilo),' records')

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/ohmBorrowedSilo_piped.csv file saved, 2  records


********************************************************************
********************************************************************
SILO COLLATERAL DEPOSITS

In [26]:
oldSiloDepositsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmDeposits.csv'
try:
    oldSiloDepositsDb = pd.read_csv (siloFileName)
except:
    oldSiloDepositsDb = pd.DataFrame()
oldSiloDepositsDb.head(100)

""


In [27]:
maxSiloDepositsTimestamp = 0
try:
    maxSiloDepositsTimestamp = (max(0,oldSiloDepositsDb['transaction_timestamp'].max()))-1
except:
        maxSiloDepositsTimestamp = 0
maxSiloDepositsTimestamp = int(maxSiloDepositsTimestamp)
maxSiloDepositsTimestamp

0

In [62]:
##this captures the ENTIRE list of people who deposited OHM
depositTicker = 0
depositslist = pd.DataFrame()
siloDepositsDb = pd.DataFrame()
depositListLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit = False
while exit==False:
    skipValue = (depositTicker)*(1000)
    deposits = silo.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
                ,'timestamp_gte': maxSiloDepositsTimestamp
                } #gOHM market
    )

    depositslist = sg.query_df([
        deposits.id
        ,deposits.hash
        ,deposits.blockNumber
        ,deposits.timestamp
        ,deposits.account.id
        ,deposits.market.id
        ,deposits.market.protocol.id
        ,deposits.market.rates.rate
        ,deposits.market.rates.duration
        ,deposits.market.rates.side
        ,deposits.market.rates.type
        ,deposits.market.rates.token.id
        ,deposits.asset.name
        ,deposits.asset.symbol
        ,deposits.asset.decimals
        ,deposits.asset.lastPriceUSD
        ,deposits.amount
        ,deposits.amountUSD
    ])

    siloDepositsDb=pd.concat([siloDepositsDb, depositslist])
    recordTimestamp1 = siloDepositsDb.iat[depositTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    siloDepositsDbLength = len(siloDepositsDb)
    siloDepositListLength = len(depositslist)
    recordID = siloDepositsDb.iat[depositTicker,0]
    deposit = siloDepositsDb.iat[depositTicker,1]
    print("iterations: ", depositTicker, "Lines skipped: ",skipValue, "records collected: ", siloDepositListLength, " - latest recordID: ",deposit, " ", recordID , " - deposits DB length: ", siloDepositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", depositTicker, "records collected: ", depositListLength, " - userdb length: ", depositsDbLength)
    depositTicker = depositTicker+1

    if daysAgo>daysLimit: exit =True
    if siloDepositsDbLength<1000: exit =True

siloDepositsDb.columns = siloDepositsDb.columns.str.replace("deposits", "transaction")
siloDepositsDb['event']='deposit'
siloDepositsDb.drop_duplicates
print("Done. Total Records collected: ", siloDepositsDbLength)

iterations:  0 Lines skipped:  0 records collected:  300  - latest recordID:  0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a45e1fc54156118d4bbdf0   0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a45e1fc54156118d4bbdf0-105  - deposits DB length:  300  - latest record from:  2023-02-21 09:49:35 0 days ago
Done. Total Records collected:  300


In [63]:
siloDepositsDb.head(10)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,transaction_market_rates_type,transaction_market_rates_token_id,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event
0,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,16678469,1677001775,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2756.217547,5621350750196149283,15493.665575,deposit
1,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,16678469,1677001775,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.983536,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2756.217547,5621350750196149283,15493.665575,deposit
2,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,16678469,1677001775,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,3.173277,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2756.217547,5621350750196149283,15493.665575,deposit
3,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,16678469,1677001775,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,LENDER,VARIABLE,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2756.217547,5621350750196149283,15493.665575,deposit
4,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,16678469,1677001775,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.117765,None,LENDER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2756.217547,5621350750196149283,15493.665575,deposit
5,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,16678469,1677001775,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,2.013938,None,LENDER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2756.217547,5621350750196149283,15493.665575,deposit
6,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2756.217547,48922199130466954750,138354.126796,deposit
7,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.983536,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2756.217547,48922199130466954750,138354.126796,deposit
8,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,3.173277,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c

In [64]:
siloDepositsDb['event'] ='deposit'
siloDepositsDb['transactionReadableAmount'] = siloDepositsDb['transaction_amount']/(10**(siloDepositsDb['transaction_asset_decimals']))
siloDepositsDb['transactionReadableAmount']=siloDepositsDb['transactionReadableAmount'].astype('float')
siloDepositsDb['dateTime'] =(pd.to_datetime(siloDepositsDb['transaction_timestamp'],unit='s'))


#we only want deposits in GOHM from those that borrowed, so drop anyone that is not on that side
siloDepositsDb.drop(siloDepositsDb.loc[siloDepositsDb['transaction_market_rates_side']!='BORROWER'].index, inplace=True)
siloDepositsDb.drop(siloDepositsDb.loc[siloDepositsDb['transaction_asset_symbol']!='gOHM'].index, inplace=True)


siloDepositEventsDbTimedClean = siloDepositsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloDepositEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])
siloDepositEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloDepositEventsDbTimedClean['calDate'] = siloDepositEventsDbTimedClean['dateTime'].dt.normalize()
siloDepositEventsDbTimedClean.head(1000)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,deposit,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,16678469,1677001775,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,5.621351,2756.217547,15493.665575,2023-02-21 17:49:35,2023-02-21
6,deposit,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,48.922199,2756.217547,138354.126796,2023-02-17 22:34:23,2023-02-17
12,deposit,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,16641740,1676555939,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,50.749992,2756.217547,145833.530065,2023-02-16 13:58:59,2023-02-16
36,deposit,0xe5163797f0922e4bded89126a213296a8c3d270a39e2...,16627499,1676383883,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.180346,2756.217547,490.375269,2023-02-14 14:11:23,2023-02-14
42,deposit,0xcc5e09e1e5c403b6925fa25f0410388de72ef1b45f30...,16627469,1676383523,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.663108,2756.217547,1803.040025,2023-02-14 14:05:23,2023-02-14
48,deposit,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.892669,2756.217547,2431.527276,2023-02-13 23:18:59,2023-02-13
54,deposit,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,16623049,1676330087,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,1.087799,2756.217547,2963.036715,2023-02-13 23:14:47,2023-02-13
66,deposit,0x17d879ec97730987652b5bcb00aaf6115b7dd232aaef...,16619969,1676292887,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,1.812154,2756.217547,4846.273689,2023-02-13 12:54:47,2023-02-13
72,deposit,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.137814,2756.217547,363.447391,2023-02-10 22:58:11,2023-02-10
84,deposit,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,9.990000,2756.217547,26839.810526,2023-02-09 10:34:59,2023-02-09


In [65]:
path =file+'/siloFinanceGohmDeposits.csv'
siloDepositEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmDeposits_piped.csv'
siloDepositEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloDepositEventsDbTimedClean),' records')

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/siloFinanceGohmDeposits_piped.csv file saved, 34  records


In [66]:
totalDeposited = sum(siloDepositEventsDbTimedClean['transactionReadableAmount'])
totalUSDDeposited = sum(siloDepositEventsDbTimedClean['transaction_amountUSD'])
print('There has been a total of',round(totalDeposited,2),  'gOHM deposited cumulatively as collateral  to date, about USD$',totalUSDDeposited)

There has been a total of 370.1 gOHM deposited cumulatively as collateral  to date, about USD$ 992163.0631162454


********************************************************************
********************************************************************
Now we need withdrawals to counter the deposits and see current state

In [67]:
oldSiloWithdrawsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmWithdraws.csv'
try:
    oldSiloWithdrawsDb = pd.read_csv (siloFileName)
except:
    oldSiloWithdrawsDb = pd.DataFrame()
oldSiloWithdrawsDb.head(100)

""


In [68]:
maxSiloWithdrawsTimestamp = 0
try:
    maxSiloWithdrawsTimestamp = (max(0,oldSiloWithdrawsDb['transaction_timestamp'].max()))-1
except:
        maxSiloWithdrawsTimestamp = 0
maxSiloWithdrawsTimestamp = int(maxSiloWithdrawsTimestamp)
maxSiloWithdrawsTimestamp

0

In [69]:
##this captures the ENTIRE list of people who withdrawed OHM
withdrawTicker = 0
withdrawslist = pd.DataFrame()
siloWithdrawsDb = pd.DataFrame()
withdrawListLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit = False
while exit==False:
    skipValue = (withdrawTicker)*(1000)
    withdraws = silo.Query.withdraws(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
                ,'timestamp_gte': maxSiloWithdrawsTimestamp
                } #gOHM market
    )

    withdrawslist = sg.query_df([
        withdraws.id
        ,withdraws.hash
        ,withdraws.blockNumber
        ,withdraws.timestamp
        ,withdraws.account.id
        ,withdraws.market.id
        ,withdraws.market.protocol.id
        ,withdraws.market.rates.rate
        ,withdraws.market.rates.duration
        ,withdraws.market.rates.side
        ,withdraws.market.rates.type
        ,withdraws.market.rates.token.id
        ,withdraws.asset.name
        ,withdraws.asset.symbol
        ,withdraws.asset.decimals
        ,withdraws.asset.lastPriceUSD
        ,withdraws.amount
        ,withdraws.amountUSD
    ])

    siloWithdrawsDb=pd.concat([siloWithdrawsDb, withdrawslist])
    recordTimestamp1 = siloWithdrawsDb.iat[withdrawTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    silowithdrawsDbLength = len(siloWithdrawsDb)
    silowithdrawListLength = len(withdrawslist)
    recordID = siloWithdrawsDb.iat[withdrawTicker,0]
    withdraw = siloWithdrawsDb.iat[withdrawTicker,1]
    print("iterations: ", withdrawTicker, "Lines skipped: ",skipValue, "records collected: ", silowithdrawListLength, " - latest recordID: ",withdraw, " ", recordID , " - withdraws DB length: ", silowithdrawsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", withdrawTicker, "records collected: ", withdrawListLength, " - userdb length: ", withdrawsDbLength)
    withdrawTicker = withdrawTicker+1

    if daysAgo>daysLimit: exit =True
    if silowithdrawListLength<1000: exit =True

siloWithdrawsDb.columns = siloWithdrawsDb.columns.str.replace("withdraws", "transaction")
siloWithdrawsDb['event']='withdraw'
siloWithdrawsDb.drop_duplicates
print("Done. Total Records collected: ", silowithdrawsDbLength)

iterations:  0 Lines skipped:  0 records collected:  150  - latest recordID:  0xf1f322af9b05e4b58c43531e4148c664ffb16e6509afa500999a11009adc3175   0xf1f322af9b05e4b58c43531e4148c664ffb16e6509afa500999a11009adc3175-163  - withdraws DB length:  150  - latest record from:  2023-02-20 17:49:35 1 days ago
Done. Total Records collected:  150


In [70]:
siloWithdrawEventsDbTimedClean = pd.DataFrame()
siloWithdrawsDb['transactionReadableAmount'] = siloWithdrawsDb['transaction_amount']/(10**(siloWithdrawsDb['transaction_asset_decimals']))
siloWithdrawsDb['transactionReadableAmount']=siloWithdrawsDb['transactionReadableAmount'].astype('float')
siloWithdrawsDb['dateTime'] =(pd.to_datetime(siloWithdrawsDb['transaction_timestamp'],unit='s'))

#we only want withdrawals in GOHM from those that borrowed, so drop anyone that is not on that side
siloWithdrawsDb.drop(siloWithdrawsDb.loc[siloWithdrawsDb['transaction_market_rates_side']!='BORROWER'].index, inplace=True)
siloWithdrawsDb.drop(siloWithdrawsDb.loc[siloWithdrawsDb['transaction_asset_symbol']!='gOHM'].index, inplace=True)

siloWithdrawEventsDbTimedClean = siloWithdrawsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloWithdrawEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])
siloWithdrawEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloWithdrawEventsDbTimedClean['calDate'] = siloWithdrawEventsDbTimedClean['dateTime'].dt.normalize()
siloWithdrawsDb.head(100)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,...,transaction_market_rates_token_id,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event,transactionReadableAmount,dateTime
12,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2756.217547,25000038820033231117,71148.084004,withdraw,25.000039,2023-02-15 15:27:47
13,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.983536,None,BORROWER,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2756.217547,25000038820033231117,71148.084004,withdraw,25.000039,2023-02-15 15:27:47
14,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,3.173277,None,BORROWER,...,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2756.217547,25000038820033231117,71148.084004,withdraw,25.000039,2023-02-15 15:27:47
18,0x85b689883076aa92af0a2356f8e4084610578bb48590...,0x85b689883076aa92af0a2356f8e4084610578bb48590...,16634844,1676472599,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2756.217547,64393035133917233507,183257.358356,withdraw,64.393035,2023-02-15 14:49:59
19,0x85b689883076aa92af0a2356f8e4084610578bb48590...,0x85b689883076aa92af0a2356f8e4084610578bb48590...,16634844,1676472599,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.983536,None,BORROWER,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2756.217547,64393035133917233507,183257.358356,withdraw,64.393035,2023-02-15 14:49:59
20,0x85b689883076aa92af0a2356f8e4084610578bb48590...,0x85b689883076aa92af0a2356f8e4084610578bb48590...,16634844,1676472599,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,3.173277,None,BORROWER,...,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2756.217547,64393035133917233507,183257.358356,withdraw,64.393035,2023-02-15 14:49:59
24,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,16628161,1676391839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2756.217547,36000000000000000000,100991.276713,withdraw,36.000000,2023-02-14 16:23:59
25,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,16628161,1676391839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.983536,None,BORROWER,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2756.217547,36000000000000000000,100991.276713,withdraw,36.000000,2023-02-14 16:23:59
26,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,0xd931b79cfc34df088f325e8ad

In [71]:
path =file+'/siloFinanceGohmWithdraws.csv'
siloWithdrawEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmWithdraws_piped.csv'
siloWithdrawEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloWithdrawEventsDbTimedClean),' records')

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/siloFinanceGohmWithdraws_piped.csv file saved, 12  records


In [72]:
totalWithdrawed = sum(siloWithdrawEventsDbTimedClean['transactionReadableAmount'])
print('There has been a total of',round(totalWithdrawed,2),  'OHM withdrawn cumulatively as collateral  to date')
totalWithdrawedUSD =sum(siloWithdrawEventsDbTimedClean['transaction_amountUSD'])
print('That is a value of USD$',round(totalWithdrawedUSD,2))

There has been a total of 182.18 OHM withdrawn cumulatively as collateral  to date
That is a value of USD$ 502805.15


In [73]:
siloCollateralBalance = totalDeposited -(totalWithdrawed)
print(totalDeposited, ' - ',totalWithdrawed,'=', siloCollateralBalance, 'currently deposited as collateral in Silo' )

370.10253752745757  -  182.17769420239753 = 187.92484332506004 currently deposited as collateral in Silo


********************************************************************
********************************************************************
VESTA BORROWS - Their API is poo RN

In [74]:
#https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance
vesta = sg.load_api('https://api.thegraph.com/subgraphs/name/shinitakunai/vestafinance-v1_2')

In [75]:
oldVestaBorrowsDb = pd.DataFrame()
siloFileName = file+'/vesta_borrow_events.csv'
try:
    oldVestaBorrowsDb = pd.read_csv (siloFileName)
except:
    oldVestaBorrowsDb = pd.DataFrame()
oldVestaBorrowsDb.head(100)

,troveChanges_id,troveChanges_transaction_id,troveChanges_transaction_blockNumber,troveChanges_debtChange,troveChanges_transaction_timestamp,troveChanges_trove_owner_id,troveChanges_trove_asset_id,troveChanges_trove_asset_name,event,dateTime
0,207939,0x7ff58296a29365afb72fd531d3ab3f893250da5af9c0...,63127102,300.000000,1676990846,0xf8a2c6db32b73f762d0f6abc136a20b3f4ccd004,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-21 14:47:26
1,207855,0x26622d7881d1e12cfa47c0572870b43f5210c31c3bfc...,62955356,2.994226,1676947803,0x6487d336bdec94d4714a4d217b15aa5c7b9dfa99,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-21 02:50:03
2,207753,0x0c398e38095275bae5c59a56645fb91799f9c656226c...,62733398,53231.980487,1676890169,0xa80191fca50be00f8952c69232c93d57eeacaf6f,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-20 10:49:29
3,207736,0x7c52f11b87402232bb8080a1d07dfcad762fb9f286fb...,62701434,2000.000000,1676882228,0x08c6f0898bf40a9899595304039247b1deeaaf9c,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-20 08:37:08
4,207597,0xa80709a7c7427369c21176d843db524c89fcddebb590...,62485311,1387.031010,1676824747,0xece24f3e222ec85738ecee770bdff74b8f6d7245,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-19 16:39:07
...,...,...,...,...,...,...,...,...,...,...
95,194161,0x3bb8d98479b1be2e4a6a894b5b7d478a1fa7e19931e6...,52529856,77.999923,1673521814,0x9abcd812bffc1ef2e81a2dde75b5ed8809289d9d,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-01-12 11:10:14
96,189406,0x31d911f11cfd60971ba876c1e475a603b3abfa02df94...,52370275,20419.210409,1673462858,0xf3821b8b4dbe47f4499c88538b5732ddb26c6c0e,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-01-11 18:47:38
97,189163,0xf6626640eca3e64db8656d01a2f3e40ab53317d21313...,52363075,1000.999760,1673460519,0x6487d336bdec94d4714a4d217b15aa5c7b9dfa99,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-01-11 18:08:39
98,184438,0x89b6ceb5723ea6958f0cf65e4d697999a2fe0236026f...,52076406,456.410463,1673357696,0x128e040082f03bbd414d966f8e6e851cc430c643,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-01-10 13:34:56


In [76]:
maxVestaBorrowTimestamp = 0
try:
    maxVestaTroveChangeId = (max(0,oldVestaBorrowsDb['troveChanges_id'].max()))
    maxVestaTroveChangeId = int(maxVestaTroveChangeId)
    vestaBorrows = vesta.Query.troveChanges(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                ,'debtChange_lt':0
                ,'id_gte': maxVestaTroveChangeId
         }
        )
    vestaBorrowslist = sg.query_df([
            vestaBorrows.id
            ,vestaBorrows.transaction.id
            ,vestaBorrows.transaction.blockNumber
            ,vestaBorrows.debtChange
            ,vestaBorrows.transaction.timestamp
            ,vestaBorrows.trove.owner.id
            ,vestaBorrows.trove.asset.id
            ,vestaBorrows.trove.asset.name
        ])
except:
    maxVestaTroveChangeId = 0
    vestaBorrows = vesta.Query.troveChanges(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                ,'debtChange_lt':0
               #,'id_gte': maxVestaTroveChangeId
         }
    )
    vestaBorrowslist = sg.query_df([
            vestaBorrows.id
            ,vestaBorrows.transaction.id
            ,vestaBorrows.transaction.blockNumber
            ,vestaBorrows.debtChange
            ,vestaBorrows.transaction.timestamp
            ,vestaBorrows.trove.owner.id
            ,vestaBorrows.trove.asset.id
            ,vestaBorrows.trove.asset.name
        ])

vestaBorrowslist.head(10)

,troveChanges_id,troveChanges_transaction_id,troveChanges_transaction_blockNumber,troveChanges_debtChange,troveChanges_transaction_timestamp,troveChanges_trove_owner_id,troveChanges_trove_asset_id,troveChanges_trove_asset_name
0,99981,0xf935941fd9f2518fa2a5f5f8d29dfa38c92fccb25b75...,34713629,-0.100000,1667397544,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
1,99820,0x467b6aa4a3f3d45e9316a143c45bf91c83ed1f662a4b...,34701032,-0.010000,1667393421,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
2,9972,0xfd781bb00f9e6d9599ce7aebda95979bfb902d3d73d2...,11845437,-2848.848834,1652313237,0x0c20e9958e4eb2d9c112d1469e17ab6c830cde09,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
3,99669,0xe104d6896bf784009bd9aa410f077f9c0a25154ab628...,34691865,-0.100000,1667390188,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
4,9962,0x2fde74916c909648c83e2725fecdc902181371a58c1f...,11842359,-700.000000,1652309981,0xe9f6711b7c92581856e0419399bc4fb9c36ec7e3,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
5,9961,0x1ebcd76a95f023492880bf7f7fbf3183e2197dd266a2...,11841593,-156.000000,1652309386,0x20fd6151bb6c78c6b5c89859bd5076dd852cc461,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
6,99531,0xc4558983947cedeace28d691cf9a70d871256ccd9021...,34681251,-0.020000,1667386332,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
7,9924,0x6a478fd975875045387ad810583e8270a25530599a6d...,11835231,-4000.003949,1652304099,0x6688b44ff874e246c441397d60fc169d4972a1ab,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
8,99043,0x24f693681f80f12946d463995c424c41f37a01dbd227...,34647273,-31300.004280,1667374762,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM
9,9896,0xdf29dc3054d53c0c8681ba1e5a9ab72d0dafbcf77de4...,11831764,-222.000000,1652301736,0x7ff15d6cf1a7bf0efa7435543cf086835fc80d53,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM


vestaBorrows = vesta.Query.troveChanges(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                ,'debtChange_lt':0
                ,'id_gte': maxVestaTroveChangeId
         }
    )
vestaBorrowslist = sg.query_df([
        vestaBorrows.id
        ,vestaBorrows.transaction.id
        ,vestaBorrows.transaction.blockNumber
        ,vestaBorrows.debtChange
        ,vestaBorrows.transaction.timestamp
        ,vestaBorrows.trove.owner.id
        ,vestaBorrows.trove.asset.id
        ,vestaBorrows.trove.asset.name
    ])

vestaBorrowslist.head(10)

In [77]:
##this captures the list of people who vestaBorrowed XAI against gOHM
vestaBorrowTicker = 0
vestaBorrowslist = pd.DataFrame()
vestaBorrowsDb = pd.DataFrame()
vestaBorrowListLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaBorrowTicker)*(1000)
    try:
        maxVestaTroveChangeId = (max(0,oldVestaBorrowsDb['troveChanges_id'].max()))
        maxVestaTroveChangeId = int(maxVestaTroveChangeId)
        vestaBorrows = vesta.Query.troveChanges(
            orderBy='id',
            orderDirection='desc',
            first=1000,
            skip = skipValue,
            where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                    ,'debtChange_gt':0
                    ,'id_gte': maxVestaTroveChangeId
             }
            )
        vestaBorrowslist = sg.query_df([
                vestaBorrows.id
                ,vestaBorrows.transaction.id
                ,vestaBorrows.transaction.blockNumber
                ,vestaBorrows.debtChange
                ,vestaBorrows.transaction.timestamp
                ,vestaBorrows.trove.owner.id
                ,vestaBorrows.trove.asset.id
                ,vestaBorrows.trove.asset.name
            ])
    except:
        maxVestaTroveChangeId = 0
        vestaBorrows = vesta.Query.troveChanges(
            orderBy='id',
            orderDirection='desc',
            first=1000,
            skip = skipValue,
            where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                    ,'debtChange_gt':0
                   #,'id_gte': maxVestaTroveChangeId
             }
        )
        vestaBorrowslist = sg.query_df([
                vestaBorrows.id
                ,vestaBorrows.transaction.id
                ,vestaBorrows.transaction.blockNumber
                ,vestaBorrows.debtChange
                ,vestaBorrows.transaction.timestamp
                ,vestaBorrows.trove.owner.id
                ,vestaBorrows.trove.asset.id
                ,vestaBorrows.trove.asset.name
            ])

    #vestaBorrowslist.sort_values(by=['vestaBorrows_timestamp'])

    vestaBorrowsDb=pd.concat([vestaBorrowsDb, vestaBorrowslist])
    recordTimestamp1 = vestaBorrowsDb.iat[vestaBorrowTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaBorrowsDbLength = len(vestaBorrowsDb)
    vestaBorrowListLength = len(vestaBorrowslist)
    recordID = vestaBorrowsDb.iat[vestaBorrowTicker,0]
    vestaBorrow = vestaBorrowsDb.iat[vestaBorrowTicker,1]
    print("iterations: ", vestaBorrowTicker, "Lines skipped: ",skipValue, "records collected: ", vestaBorrowListLength, " - latest recordID: ",vestaBorrow, " ", recordID , " - vestaBorrows DB length: ", vestaBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaBorrowTicker, "records collected: ", vestaBorrowListLength, " - userdb length: ", vestaBorrowsDbLength)
    vestaBorrowTicker = vestaBorrowTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaBorrowListLength<1000: exit =True

#vestaBorrowsDbTimed.columns = vestaBorrowsDbTimed.columns.str.replace("borrows", "transaction")
vestaBorrowsDb['event']='borrow'
print("Done. Total Records collected: ", vestaBorrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  0x21da8f0687a20a391e7e08c8af41efd288af11da388165d1fe8423ff8468ed59   9966  - vestaBorrows DB length:  1000  - latest record from:  2022-05-11 16:41:54 286 days ago
Done. Total Records collected:  1000


In [78]:
vestaBorrowsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(troveChanges_transaction_timestamp::int) dateTime "
               "From vestaBorrowsDb "
                               "order by troveChanges_transaction_timestamp desc"
               ).df()

vestaBorrowsDbTimed.head(1000)

,troveChanges_id,troveChanges_transaction_id,troveChanges_transaction_blockNumber,troveChanges_debtChange,troveChanges_transaction_timestamp,troveChanges_trove_owner_id,troveChanges_trove_asset_id,troveChanges_trove_asset_name,event,dateTime
0,207939,0x7ff58296a29365afb72fd531d3ab3f893250da5af9c0...,63127102,300.000000,1676990846,0xf8a2c6db32b73f762d0f6abc136a20b3f4ccd004,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-21 14:47:26
1,207855,0x26622d7881d1e12cfa47c0572870b43f5210c31c3bfc...,62955356,2.994226,1676947803,0x6487d336bdec94d4714a4d217b15aa5c7b9dfa99,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-21 02:50:03
2,207753,0x0c398e38095275bae5c59a56645fb91799f9c656226c...,62733398,53231.980487,1676890169,0xa80191fca50be00f8952c69232c93d57eeacaf6f,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-20 10:49:29
3,207736,0x7c52f11b87402232bb8080a1d07dfcad762fb9f286fb...,62701434,2000.000000,1676882228,0x08c6f0898bf40a9899595304039247b1deeaaf9c,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-20 08:37:08
4,207597,0xa80709a7c7427369c21176d843db524c89fcddebb590...,62485311,1387.031010,1676824747,0xece24f3e222ec85738ecee770bdff74b8f6d7245,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-19 16:39:07
...,...,...,...,...,...,...,...,...,...,...
995,321,0xe7a2f919a8b75cdfda3826f235fb6fb09d7fe3a285d2...,5697588,2231.175000,1644385430,0x1cb4831b6df1d9cb07279b93a0e101db9b2295d0,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2022-02-09 05:43:50
996,313,0x8a69f94169d23f37811def09cee4008e7305b538a748...,5697498,12180.000000,1644385318,0xf5b4635f3858a2bc80a17bd4450e05a35ad99a72,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2022-02-09 05:41:58
997,312,0x3cf08c9c4d3cf6c6a5bfd9bc707237a199ac0eb0559c...,5697463,333.750000,1644385225,0x87209dc4b76b14b67bc5e5e5c0737e7d002a219c,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2022-02-09 05:40:25
998,305,0x54c221378376ddc4b488cf03648bbc0a83ad6f51ca9f...,5697391,1567.755688,1644385147,0x56c6e3385acc23826ef764066fa5a5f422c78e14,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2022-02-09 05:39:07


In [79]:
vestaBorrowsDbTimed = pd.concat([oldVestaBorrowsDb,vestaBorrowsDbTimed]).drop_duplicates().reset_index(drop=True)
vestaBorrowsDbTimed.drop_duplicates()
vestaBorrowsDbTimed = db.query("select "
               "    * "
               "From vestaBorrowsDbTimed "
                         "group by 1,2,3,4,5,6,7,8,9,10 "
                     # "order by borrows_amountUSD desc "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaBorrowsDbTimed.head(100)

,troveChanges_id,troveChanges_transaction_id,troveChanges_transaction_blockNumber,troveChanges_debtChange,troveChanges_transaction_timestamp,troveChanges_trove_owner_id,troveChanges_trove_asset_id,troveChanges_trove_asset_name,event,dateTime
0,207939,0x7ff58296a29365afb72fd531d3ab3f893250da5af9c0...,63127102,300.000000,1676990846,0xf8a2c6db32b73f762d0f6abc136a20b3f4ccd004,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-21 14:47:26
1,207855,0x26622d7881d1e12cfa47c0572870b43f5210c31c3bfc...,62955356,2.994226,1676947803,0x6487d336bdec94d4714a4d217b15aa5c7b9dfa99,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-21 02:50:03
2,207753,0x0c398e38095275bae5c59a56645fb91799f9c656226c...,62733398,53231.980487,1676890169,0xa80191fca50be00f8952c69232c93d57eeacaf6f,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-20 10:49:29
3,207736,0x7c52f11b87402232bb8080a1d07dfcad762fb9f286fb...,62701434,2000.000000,1676882228,0x08c6f0898bf40a9899595304039247b1deeaaf9c,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-20 08:37:08
4,207597,0xa80709a7c7427369c21176d843db524c89fcddebb590...,62485311,1387.031010,1676824747,0xece24f3e222ec85738ecee770bdff74b8f6d7245,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-02-19 16:39:07
...,...,...,...,...,...,...,...,...,...,...
95,189406,0x31d911f11cfd60971ba876c1e475a603b3abfa02df94...,52370275,20419.210409,1673462858,0xf3821b8b4dbe47f4499c88538b5732ddb26c6c0e,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-01-11 18:47:38
96,189163,0xf6626640eca3e64db8656d01a2f3e40ab53317d21313...,52363075,1000.999760,1673460519,0x6487d336bdec94d4714a4d217b15aa5c7b9dfa99,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-01-11 18:08:39
97,184438,0x89b6ceb5723ea6958f0cf65e4d697999a2fe0236026f...,52076406,456.410463,1673357696,0x128e040082f03bbd414d966f8e6e851cc430c643,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-01-10 13:34:56
98,184231,0xfbe0647202dc36327ae5cade16917a72c7df874fa99a...,51924666,923.036538,1673299254,0xf7b18e107eb36797f4ce36de756630b9c30969ad,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,borrow,2023-01-09 21:20:54


In [80]:
path =file+'/vesta_borrow_events.csv'
vestaBorrowsDbTimed.to_csv(path, index = False)

path =file+'/vesta_borrow_events_piped.csv'
vestaBorrowsDbTimed.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(vestaBorrowsDbTimed),' records')

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/vesta_borrow_events_piped.csv file saved, 1137  records


************************************************************
FINE.... FRAX TOO

This subgraph is completely different, so we're going to  have to crunch in order to merge all these tables into coherent reports

In [95]:
frax = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/GELTrjPJYEzxyp6Y2CtEaYpGHcJNrJA6i5Ci4KfJSEsf')
#frax = sg.load_api('https://api.thegraph.com/subgraphs/name/frax-finance-data/fraxlend-subgraph---mainnet')

In [96]:
oldFraxActions = pd.DataFrame()
fraxFileName = file+'/frax_actions.csv'
try:
    oldFraxActions = pd.read_csv (fraxFileName)
except:
    oldFraxActions = pd.DataFrame()
oldFraxActions.head(1000)

""


In [97]:
oldFraxActions = pd.DataFrame()
fraxFileName = file+'/frax_borrows.csv'
try:
    oldFraxActions = pd.read_csv (fraxFileName)
except:
    oldFraxActions = pd.DataFrame()
oldFraxActions.head(1000)

""


In [98]:
oldFraxDeposits = pd.DataFrame()
fraxFileName = file+'/frax_deposits.csv'
try:
    oldFraxActions = pd.read_csv (fraxFileName)
except:
    oldFraxActions = pd.DataFrame()
oldFraxActions.head(1000)

""


In [99]:
oldFraxWithdraws = pd.DataFrame()
fraxFileName = file+'/frax_deposits.csv'
try:
    oldFraxActions = pd.read_csv (fraxFileName)
except:
    oldFraxActions = pd.DataFrame()
oldFraxWithdraws.head(1000)

""


In [100]:
maxActionFraxTimestamp = 0
try:
    maxActionFraxTimestamp = (max(1,oldFraxActions['actions_timestamp'].max()))-1
except:
        maxActionFraxTimestamp = 0
maxActionFraxTimestamp = int(maxActionFraxTimestamp)
maxActionFraxTimestamp

0

In [101]:
actionsQuery = frax.Query.actions(
     orderBy='timestamp'
    ,orderDirection='desc'
    ,first=1000
    #,where = [frax.Query.actions.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
    ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52'
             ,'timestamp_gte': maxBorrowFraxTimestamp
             }
    #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
#{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

)
actions= sg.query_df([
    actionsQuery.id
    ,actionsQuery.type
    ,actionsQuery.amount
    ,actionsQuery.timestamp
    ,actionsQuery.block
    ,actionsQuery.token.name
    ,actionsQuery.root.address
    ,actionsQuery.pair.asset.symbol
    ,actionsQuery.pair.asset.decimals
    ,actionsQuery.pair.collateral.symbol
    ,actionsQuery.pair.collateral.decimals
    ]
)
actions['dateTime'] =(pd.to_datetime(actions['actions_timestamp'],unit='s'))
actions.head(100)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,dateTime
0,0x4437b802baa713bfa7f02667996787221356f632b1e9...,borrowAsset,2000000000000000000000,1677000779,16678387,Frax,0xf2ba9e4872f03a72a94f8c32e87fc0f2981e19a8,FRAX,18,gOHM,18,2023-02-21 17:32:59
1,0x4437b802baa713bfa7f02667996787221356f632b1e9...,addCollateral,9933187141283871315,1677000779,16678387,Governance OHM,0xf2ba9e4872f03a72a94f8c32e87fc0f2981e19a8,FRAX,18,gOHM,18,2023-02-21 17:32:59


In [102]:
actionListLength=0
actionTicker = 0
fraxUsersDb = pd.DataFrame()
actionsDb = pd.DataFrame()
actionListLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit = False
while exit==False:
    #listLength=0
    skipValue = (actionTicker)*(1000)
    actionsQuery = frax.Query.actions(
         orderBy='timestamp'
        ,orderDirection='desc'
        ,first=1000
        ,skipValue = skipValue
        #,where = [frax.Query.actions.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
        ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52'
                 ,'timestamp_gte': maxBorrowFraxTimestamp
                 }
        #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
    #{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

    )
    actions= sg.query_df([
        actionsQuery.id
        ,actionsQuery.type
        ,actionsQuery.amount
        ,actionsQuery.timestamp
        ,actionsQuery.block
        ,actionsQuery.token.name
        ,actionsQuery.root.address
        ,actionsQuery.pair.asset.symbol
        ,actionsQuery.pair.asset.decimals
        ,actionsQuery.pair.collateral.symbol
        ,actionsQuery.pair.collateral.decimals
        ]
    )

    actions.sort_values(by=['actions_timestamp'])

    actionsDb=pd.concat([actionsDb, actions])
    recordTimestamp1 = actionsDb.iat[actionTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    actionsDbLength = len(actionsDb)
    actionListLength = len(actions)
    recordID = actionsDb.iat[actionTicker,0]
    action = actionsDb.iat[actionTicker,1]
    print("iterations: ", actionTicker, "Lines skipped: ",skipValue, "records collected: ", actionListLength, " - latest recordID: ",action, " ", recordID , " - actions DB length: ", actionsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", actionTicker, "records collected: ", actionListLength, " - userdb length: ", actionsDbLength)
    if daysAgo>daysLimit: exit =True
    if actionListLength<1000: exit =True
    actionTicker = actionTicker+1
actionsDb['DateTime']=(pd.to_datetime(actionsDb['actions_timestamp'],unit='s'))
print("Done. Total Records collected: ", actionsDbLength)

iterations:  0 Lines skipped:  0 records collected:  2  - latest recordID:  borrowAsset   0x4437b802baa713bfa7f02667996787221356f632b1e96879f14c493c69b98fed+1  - actions DB length:  2  - latest record from:  2023-02-21 09:32:59 0 days ago
Done. Total Records collected:  2


In [103]:
actionsDb['transactionReadableAmount'] = actionsDb['actions_amount']/(10**(actionsDb['actions_pair_asset_decimals']))
actionsDb['transactionReadableAmount']=actionsDb['transactionReadableAmount'].astype('float')
actionsDb.head(2000)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,DateTime,transactionReadableAmount
0,0x4437b802baa713bfa7f02667996787221356f632b1e9...,borrowAsset,2000000000000000000000,1677000779,16678387,Frax,0xf2ba9e4872f03a72a94f8c32e87fc0f2981e19a8,FRAX,18,gOHM,18,2023-02-21 17:32:59,2000.000000
1,0x4437b802baa713bfa7f02667996787221356f632b1e9...,addCollateral,9933187141283871315,1677000779,16678387,Governance OHM,0xf2ba9e4872f03a72a94f8c32e87fc0f2981e19a8,FRAX,18,gOHM,18,2023-02-21 17:32:59,9.933187


In [104]:
fraxActionsDb= pd.DataFrame()
fraxActionsDbTimed =pd.concat([actionsDb, oldFraxActions])
fraxActionsDbTimed.head(1000)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,DateTime,transactionReadableAmount
0,0x4437b802baa713bfa7f02667996787221356f632b1e9...,borrowAsset,2000000000000000000000,1677000779,16678387,Frax,0xf2ba9e4872f03a72a94f8c32e87fc0f2981e19a8,FRAX,18,gOHM,18,2023-02-21 17:32:59,2000.000000
1,0x4437b802baa713bfa7f02667996787221356f632b1e9...,addCollateral,9933187141283871315,1677000779,16678387,Governance OHM,0xf2ba9e4872f03a72a94f8c32e87fc0f2981e19a8,FRAX,18,gOHM,18,2023-02-21 17:32:59,9.933187


In [105]:
fraxActionsDbTimedLength = len(fraxActionsDbTimed)
path =file+'/frax_borrows.csv'
fraxActionsDbTimed.to_csv(path, index = False)

path =file+'/frax_borrows_piped.csv'
fraxActionsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', fraxActionsDbTimedLength,' records at', (dt.datetime.now()) )


/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/frax_borrows_piped.csv file saved, 2  records at 2023-02-21 11:38:27.439032


In [106]:
fraxActionsDbTimed.head(10)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,DateTime,transactionReadableAmount
0,0x4437b802baa713bfa7f02667996787221356f632b1e9...,borrowAsset,2000000000000000000000,1677000779,16678387,Frax,0xf2ba9e4872f03a72a94f8c32e87fc0f2981e19a8,FRAX,18,gOHM,18,2023-02-21 17:32:59,2000.000000
1,0x4437b802baa713bfa7f02667996787221356f632b1e9...,addCollateral,9933187141283871315,1677000779,16678387,Governance OHM,0xf2ba9e4872f03a72a94f8c32e87fc0f2981e19a8,FRAX,18,gOHM,18,2023-02-21 17:32:59,9.933187


In [107]:
#we might as well create all the DBs here and save ourselves time and grief
fraxBorrowsDbTimed = pd.DataFrame()
fraxBorrowsDbTimed = db.query("Select distinct * from fraxActionsDbTimed where actions_type = 'borrowAsset' and actions_pair_collateral_symbol= 'gOHM' and right(actions_id,2)<>'+1' ").df()

fraxDepositsDbTimed = pd.DataFrame()
fraxDepositsDbTimed =fraxBorrows = db.query("Select distinct * from fraxActionsDbTimed where actions_type = 'addCollateral' and actions_pair_collateral_symbol= 'gOHM' and right(actions_id,2)<>'+1' ").df()

fraxWithdrawsDbTimed = pd.DataFrame()
fraxWithdrawsDbTimed = db.query("Select distinct * from fraxActionsDbTimed where actions_type = 'removeCollateral' and actions_pair_collateral_symbol= 'gOHM' and right(actions_id,2)<>'+1' ").df()

In [108]:
fraxBorrowsDbTimed.head(1000)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,DateTime,transactionReadableAmount


In [109]:
fraxDepositsDbTimed.head(1000)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,DateTime,transactionReadableAmount
0,0x4437b802baa713bfa7f02667996787221356f632b1e9...,addCollateral,9.933187e+18,1677000779,16678387,Governance OHM,0xf2ba9e4872f03a72a94f8c32e87fc0f2981e19a8,FRAX,18,gOHM,18,2023-02-21 17:32:59,9.933187


In [110]:
fraxWithdrawsDbTimed.head(10)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,DateTime,transactionReadableAmount


In [111]:
#fraxActionsDbTimed = pd.concat([oldFraxBorrows,fraxBorrowsDbTimed]).drop_duplicates().reset_index(drop=True)
fraxActionsDbTimed.sort_values(['actions_id','actions_type','actions_amount','actions_timestamp','actions_block','actions_token_name','actions_root_address','DateTime'])
fraxActionsDbTimed.drop_duplicates(keep='first' )
fraxActionsDbTimed.head(100)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,DateTime,transactionReadableAmount
0,0x4437b802baa713bfa7f02667996787221356f632b1e9...,borrowAsset,2000000000000000000000,1677000779,16678387,Frax,0xf2ba9e4872f03a72a94f8c32e87fc0f2981e19a8,FRAX,18,gOHM,18,2023-02-21 17:32:59,2000.000000
1,0x4437b802baa713bfa7f02667996787221356f632b1e9...,addCollateral,9933187141283871315,1677000779,16678387,Governance OHM,0xf2ba9e4872f03a72a94f8c32e87fc0f2981e19a8,FRAX,18,gOHM,18,2023-02-21 17:32:59,9.933187


In [112]:
fraxActionsDbTimedLength = len(fraxActionsDbTimed)
path =file+'/frax_actions.csv'
fraxActionsDbTimed.to_csv(path, index = False)

path =file+'/frax_actions_piped.csv'
fraxActionsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', fraxActionsDbTimedLength,' records at', (dt.datetime.now()) )

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/frax_actions_piped.csv file saved, 2  records at 2023-02-21 11:38:27.546916


In [113]:
fraxBorrowsDbTimedLength = len(fraxBorrowsDbTimed)
path =file+'/frax_borrows.csv'
fraxBorrowsDbTimed.to_csv(path, index = False)

path =file+'/frax_borrows_piped.csv'
fraxBorrowsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', fraxActionsDbTimedLength,' records at', (dt.datetime.now()) )

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/frax_borrows_piped.csv file saved, 2  records at 2023-02-21 11:38:27.554153


In [114]:
fraxDepositsDbTimedLength = len(fraxBorrowsDbTimed)
path =file+'/frax_deposits.csv'
fraxBorrowsDbTimed.to_csv(path, index = False)

path =file+'/frax_deposits_piped.csv'
fraxBorrowsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', fraxActionsDbTimedLength,' records at', (dt.datetime.now()) )

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/frax_deposits_piped.csv file saved, 2  records at 2023-02-21 11:38:27.561529


In [115]:
fraxWithdrawsDbTimedLength = len(fraxBorrowsDbTimed)
path =file+'/frax_borrows.csv'
fraxWithdrawsDbTimed.to_csv(path, index = False)

path =file+'/frax_borrows_piped.csv'
fraxWithdrawsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', fraxWithdrawsDbTimedLength,' records at', (dt.datetime.now()) )

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/frax_borrows_piped.csv file saved, 0  records at 2023-02-21 11:38:27.568862


Collateral check

In [116]:
totalgOHMDepositedFrax = sum(fraxDepositsDbTimed['transactionReadableAmount'])
print('There has been a total of',round(totalgOHMDepositedFrax,2),  'gOHM deposited cumulatively as collateral  to date')

There has been a total of 9.93 gOHM deposited cumulatively as collateral  to date


In [117]:
totalgOHMWithdrawnFrax = sum(fraxWithdrawsDbTimed['transactionReadableAmount'])
print('There has been a total of',round(totalgOHMWithdrawnFrax,2),  'gOHM withdrawn cumulatively as collateral  to date')

There has been a total of 0 gOHM withdrawn cumulatively as collateral  to date


In [118]:
collateralBalance = totalgOHMDepositedFrax-totalgOHMWithdrawnFrax
print(totalgOHMDepositedFrax, ' - ',totalgOHMWithdrawnFrax,'=', collateralBalance, 'currently deposited as collateral in Silo' )

9.933187141283872  -  0 = 9.933187141283872 currently deposited as collateral in Silo


**********************************************************
MERGE DIFFERENT DBs

In [119]:
#COLLATERALDEPOSITS
fraxDepositsDbPreMerge = db.query(
    "select distinct"
    " actions_id transaction_id"
    ", actions_timestamp transaction_timestamp"
    ", DateTime dateTime"
    ", actions_root_address transaction_from "
    ", actions_pair_collateral_symbol asset_symbol"
    ", transactionReadableAmount ReadableAmount "
    "from fraxDepositsDbTimed "
).df()
fraxDepositsDbPreMerge['protocol'] = 'Frax'
fraxDepositsDbPreMerge['action'] = 'depositCollateral'
fraxDepositsDbPreMerge.head(10)



,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0x4437b802baa713bfa7f02667996787221356f632b1e9...,1677000779,2023-02-21 17:32:59,0xf2ba9e4872f03a72a94f8c32e87fc0f2981e19a8,gOHM,9.933187,Frax,depositCollateral


In [120]:
siloDepositEventsDbTimedClean.head(1)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,deposit,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,16678469,1677001775,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,5.621351,2756.217547,15493.665575,2023-02-21 17:49:35,2023-02-21


In [121]:
siloDepositsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash transaction_id "
    ", transaction_timestamp transaction_timestamp "
    ", dateTime "
    ", transaction_account_id transaction_from "
    ", transaction_asset_symbol asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from siloDepositEventsDbTimedClean "
).df()
siloDepositsDbPreMerge['protocol'] = 'silo'
siloDepositsDbPreMerge['action'] = 'depositCollateral'
siloDepositsDbPreMerge.head(10)


,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,1677001775,2023-02-21 17:49:35,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,gOHM,5.621351,silo,depositCollateral
1,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,1676673263,2023-02-17 22:34:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,gOHM,48.922199,silo,depositCollateral
2,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,1676555939,2023-02-16 13:58:59,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,gOHM,50.749992,silo,depositCollateral
3,0xe5163797f0922e4bded89126a213296a8c3d270a39e2...,1676383883,2023-02-14 14:11:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.180346,silo,depositCollateral
4,0xcc5e09e1e5c403b6925fa25f0410388de72ef1b45f30...,1676383523,2023-02-14 14:05:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.663108,silo,depositCollateral
5,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,1676330339,2023-02-13 23:18:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.892669,silo,depositCollateral
6,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,1676330087,2023-02-13 23:14:47,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,1.087799,silo,depositCollateral
7,0x17d879ec97730987652b5bcb00aaf6115b7dd232aaef...,1676292887,2023-02-13 12:54:47,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,1.812154,silo,depositCollateral
8,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,1676069891,2023-02-10 22:58:11,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,gOHM,0.137814,silo,depositCollateral
9,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,1675938899,2023-02-09 10:34:59,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,gOHM,9.990000,silo,depositCollateral


DONT FORGET VESTA WHEN IT GETS FIXED

In [122]:
gOHMCollateralDeposits = pd.concat([siloDepositsDbPreMerge,fraxDepositsDbPreMerge])
gOHMCollateralDeposits.head(1000)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,1677001775,2023-02-21 17:49:35,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,gOHM,5.621351,silo,depositCollateral
1,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,1676673263,2023-02-17 22:34:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,gOHM,48.922199,silo,depositCollateral
2,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,1676555939,2023-02-16 13:58:59,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,gOHM,50.749992,silo,depositCollateral
3,0xe5163797f0922e4bded89126a213296a8c3d270a39e2...,1676383883,2023-02-14 14:11:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.180346,silo,depositCollateral
4,0xcc5e09e1e5c403b6925fa25f0410388de72ef1b45f30...,1676383523,2023-02-14 14:05:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.663108,silo,depositCollateral
5,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,1676330339,2023-02-13 23:18:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.892669,silo,depositCollateral
6,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,1676330087,2023-02-13 23:14:47,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,1.087799,silo,depositCollateral
7,0x17d879ec97730987652b5bcb00aaf6115b7dd232aaef...,1676292887,2023-02-13 12:54:47,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,1.812154,silo,depositCollateral
8,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,1676069891,2023-02-10 22:58:11,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,gOHM,0.137814,silo,depositCollateral
9,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,1675938899,2023-02-09 10:34:59,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,gOHM,9.990000,silo,depositCollateral


In [123]:
gOHMCollateralDepositsLength = len(gOHMCollateralDeposits)
path =file+'/gOHMCollateralDeposits.csv'
gOHMCollateralDeposits.to_csv(path, index = False)

path =file+'/gOHMCollateralDeposits_piped.csv'
gOHMCollateralDeposits.to_csv(path, index = False, sep='|')

print(path, 'file saved,', gOHMCollateralDepositsLength,' records at', (dt.datetime.now()) )

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/gOHMCollateralDeposits_piped.csv file saved, 35  records at 2023-02-21 11:38:27.677667


In [124]:
#ALL WITHDRAW EVENTS

In [125]:
fraxWithdrawsDbPreMerge = db.query(
    "select distinct"
    " actions_id transaction_id"
    ", actions_timestamp transaction_timestamp"
    ", DateTime dateTime "
    ", actions_root_address transaction_from "
    ", actions_pair_collateral_symbol asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from fraxWithdrawsDbTimed "
).df()
fraxWithdrawsDbPreMerge['protocol'] = 'Frax'
fraxWithdrawsDbPreMerge['action'] = 'withdrawCollateral'
fraxWithdrawsDbPreMerge.head(1000)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action


In [126]:
siloWithdrawsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash transaction_id "
    ", transaction_timestamp transaction_timestamp "
    ", dateTime "
    ", transaction_account_id transaction_from "
    ", transaction_asset_symbol asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from siloWithdrawEventsDbTimedClean "
).df()
siloWithdrawsDbPreMerge['protocol'] = 'silo'
siloWithdrawsDbPreMerge['action'] = 'withdrawCollateral'
siloWithdrawsDbPreMerge.head(1000)


,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,1676474867,2023-02-15 15:27:47,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,gOHM,25.000039,silo,withdrawCollateral
1,0x85b689883076aa92af0a2356f8e4084610578bb48590...,1676472599,2023-02-15 14:49:59,0xb0399dac8f4d4100b49b2a7b3873481114229d18,gOHM,64.393035,silo,withdrawCollateral
2,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,1676391839,2023-02-14 16:23:59,0xb0399dac8f4d4100b49b2a7b3873481114229d18,gOHM,36.000000,silo,withdrawCollateral
3,0xd4f7aeb47e1e07903a3e9298762bc164764fffb2f973...,1676381987,2023-02-14 13:39:47,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.883213,silo,withdrawCollateral
4,0xc7193a2aa216c65fe82652e85b4adc20a60b274bd070...,1676072123,2023-02-10 23:35:23,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,gOHM,0.137814,silo,withdrawCollateral
5,0xed87a5e09b9abe135be66d51493eca516d78aaf1d4d5...,1675924055,2023-02-09 06:27:35,0xb0399dac8f4d4100b49b2a7b3873481114229d18,gOHM,19.607121,silo,withdrawCollateral
6,0xcfa1e8a102e407470f7cc479576cceab5f872c1a1377...,1675737947,2023-02-07 02:45:47,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,gOHM,0.194798,silo,withdrawCollateral
7,0xb8682e4d0d5fe1297f21ff66347debdfc75c3956612d...,1674641375,2023-01-25 10:09:35,0xee1520f94f304e8d551cbf310fe214212e3ca34a,gOHM,20.000000,silo,withdrawCollateral
8,0xcc9db9a7563b473c4b8e4ca0f4f7fb90ca709b022469...,1673230019,2023-01-09 02:06:59,0x25fdda95197f804316f5ba8dbb66ac173ee19981,gOHM,2.252766,silo,withdrawCollateral
9,0x7c24d04318da39f08c7ba4f8d413a487133ce1b1f117...,1671741731,2022-12-22 20:42:11,0x96035b45bc0d22f0ed11283a91145fb001c48370,gOHM,13.520668,silo,withdrawCollateral


DONT FORGET VESTA

In [127]:
gOHMCollateralWithdrawals = pd.concat([siloWithdrawsDbPreMerge,fraxWithdrawsDbPreMerge])
gOHMCollateralWithdrawals.head(1000)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,1676474867,2023-02-15 15:27:47,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,gOHM,25.000039,silo,withdrawCollateral
1,0x85b689883076aa92af0a2356f8e4084610578bb48590...,1676472599,2023-02-15 14:49:59,0xb0399dac8f4d4100b49b2a7b3873481114229d18,gOHM,64.393035,silo,withdrawCollateral
2,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,1676391839,2023-02-14 16:23:59,0xb0399dac8f4d4100b49b2a7b3873481114229d18,gOHM,36.000000,silo,withdrawCollateral
3,0xd4f7aeb47e1e07903a3e9298762bc164764fffb2f973...,1676381987,2023-02-14 13:39:47,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.883213,silo,withdrawCollateral
4,0xc7193a2aa216c65fe82652e85b4adc20a60b274bd070...,1676072123,2023-02-10 23:35:23,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,gOHM,0.137814,silo,withdrawCollateral
5,0xed87a5e09b9abe135be66d51493eca516d78aaf1d4d5...,1675924055,2023-02-09 06:27:35,0xb0399dac8f4d4100b49b2a7b3873481114229d18,gOHM,19.607121,silo,withdrawCollateral
6,0xcfa1e8a102e407470f7cc479576cceab5f872c1a1377...,1675737947,2023-02-07 02:45:47,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,gOHM,0.194798,silo,withdrawCollateral
7,0xb8682e4d0d5fe1297f21ff66347debdfc75c3956612d...,1674641375,2023-01-25 10:09:35,0xee1520f94f304e8d551cbf310fe214212e3ca34a,gOHM,20.000000,silo,withdrawCollateral
8,0xcc9db9a7563b473c4b8e4ca0f4f7fb90ca709b022469...,1673230019,2023-01-09 02:06:59,0x25fdda95197f804316f5ba8dbb66ac173ee19981,gOHM,2.252766,silo,withdrawCollateral
9,0x7c24d04318da39f08c7ba4f8d413a487133ce1b1f117...,1671741731,2022-12-22 20:42:11,0x96035b45bc0d22f0ed11283a91145fb001c48370,gOHM,13.520668,silo,withdrawCollateral


In [128]:
gOHMCollateralWithdrawalsLength = len(gOHMCollateralWithdrawals)
path =file+'/gOHMCollateralDeposits.csv'
gOHMCollateralWithdrawals.to_csv(path, index = False)

path =file+'/gOHMCollateralWithdrawals_piped.csv'
gOHMCollateralWithdrawals.to_csv(path, index = False, sep='|')

print(path, 'file saved,', gOHMCollateralWithdrawalsLength,' records at', (dt.datetime.now()) )

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/gOHMCollateralWithdrawals_piped.csv file saved, 12  records at 2023-02-21 11:38:27.758216


In [129]:
#BORROWS

In [130]:
fraxBorrowsDbPreMerge = db.query(
    "select distinct"
    " actions_id transaction_id"
    ", actions_timestamp transaction_timestamp"
    ", DateTime as dateTime "
    ", actions_root_address transaction_from "
    ", 'FRAX' asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from fraxBorrowsDbTimed "
).df()
fraxBorrowsDbPreMerge['protocol'] = 'Frax'
fraxBorrowsDbPreMerge['action'] = 'borrowStablecoin'
fraxBorrowsDbPreMerge.head(10)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action


In [131]:
siloBorrowsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash transaction_id "
    ", transaction_timestamp transaction_timestamp "
    ", dateTime "
    ", transaction_account_id transaction_from "
    ", 'XAI' asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from siloBorrowEventsDbTimedClean "
).df()
siloBorrowsDbPreMerge['protocol'] = 'silo'
siloBorrowsDbPreMerge['action'] = 'borrowStablecoin'
siloBorrowsDbPreMerge.head(10)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0x84778306b3f662f7d98e431591f302d924d143c3c0b2...,1677001823,2023-02-21 17:50:23,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,XAI,6540.000000,silo,borrowStablecoin
1,0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5...,1676673563,2023-02-17 22:39:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,XAI,60500.000000,silo,borrowStablecoin
2,0xd9329978828295f4f091eaa77a6d8c388c5616bbe92c...,1676590307,2023-02-16 23:31:47,0x96035b45bc0d22f0ed11283a91145fb001c48370,XAI,0.600000,silo,borrowStablecoin
3,0x3b0f26e10c63a71c916e8a68bd1889d9bab38aeb62d7...,1676556011,2023-02-16 14:00:11,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,XAI,100000.000000,silo,borrowStablecoin
4,0xfa292276fc71f90e29ca90a7b8a2be9a4094f459fb99...,1676472911,2023-02-15 14:55:11,0x16ff8ad104fa6195260a6d4a3f311c5f120df77d,XAI,3000.000000,silo,borrowStablecoin
5,0xd5dc7147192591c7d19856d12e9278928932a5921653...,1676383655,2023-02-14 14:07:35,0xc0b1f200aa9546ce074397bed7438bb2ec033877,XAI,1000.000000,silo,borrowStablecoin
6,0x39c11cb01e34d9f8ca55a8820482eaf8a217033cdb6f...,1676375459,2023-02-14 11:50:59,0x16ff8ad104fa6195260a6d4a3f311c5f120df77d,XAI,1.920000,silo,borrowStablecoin
7,0xf1d5b9c7f23bd9935061e4d08734cbfba9b9a53bacd5...,1676370563,2023-02-14 10:29:23,0x339fd7c17ce670e7821f2f2ee94ddd49771483da,XAI,4000.000000,silo,borrowStablecoin
8,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,1676330255,2023-02-13 23:17:35,0xc0b1f200aa9546ce074397bed7438bb2ec033877,XAI,2457.516074,silo,borrowStablecoin
9,0x3dba2dd3c913b399164efd71db73121c9c94c0096139...,1676328371,2023-02-13 22:46:11,0x46535cbc3fa574c32ea392d4f89c518290597072,XAI,26.400000,silo,borrowStablecoin


DONT FORGET VESTA

In [132]:
stablecoinsBorrowedWithGOHMCollateral = pd.concat([siloBorrowsDbPreMerge,fraxBorrowsDbPreMerge])
stablecoinsBorrowedWithGOHMCollateral.head(1000)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0x84778306b3f662f7d98e431591f302d924d143c3c0b2...,1677001823,2023-02-21 17:50:23,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,XAI,6540.000000,silo,borrowStablecoin
1,0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5...,1676673563,2023-02-17 22:39:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,XAI,60500.000000,silo,borrowStablecoin
2,0xd9329978828295f4f091eaa77a6d8c388c5616bbe92c...,1676590307,2023-02-16 23:31:47,0x96035b45bc0d22f0ed11283a91145fb001c48370,XAI,0.600000,silo,borrowStablecoin
3,0x3b0f26e10c63a71c916e8a68bd1889d9bab38aeb62d7...,1676556011,2023-02-16 14:00:11,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,XAI,100000.000000,silo,borrowStablecoin
4,0xfa292276fc71f90e29ca90a7b8a2be9a4094f459fb99...,1676472911,2023-02-15 14:55:11,0x16ff8ad104fa6195260a6d4a3f311c5f120df77d,XAI,3000.000000,silo,borrowStablecoin
5,0xd5dc7147192591c7d19856d12e9278928932a5921653...,1676383655,2023-02-14 14:07:35,0xc0b1f200aa9546ce074397bed7438bb2ec033877,XAI,1000.000000,silo,borrowStablecoin
6,0x39c11cb01e34d9f8ca55a8820482eaf8a217033cdb6f...,1676375459,2023-02-14 11:50:59,0x16ff8ad104fa6195260a6d4a3f311c5f120df77d,XAI,1.920000,silo,borrowStablecoin
7,0xf1d5b9c7f23bd9935061e4d08734cbfba9b9a53bacd5...,1676370563,2023-02-14 10:29:23,0x339fd7c17ce670e7821f2f2ee94ddd49771483da,XAI,4000.000000,silo,borrowStablecoin
8,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,1676330255,2023-02-13 23:17:35,0xc0b1f200aa9546ce074397bed7438bb2ec033877,XAI,2457.516074,silo,borrowStablecoin
9,0x3dba2dd3c913b399164efd71db73121c9c94c0096139...,1676328371,2023-02-13 22:46:11,0x46535cbc3fa574c32ea392d4f89c518290597072,XAI,26.400000,silo,borrowStablecoin


In [136]:
stablecoinsBorrowedWithGOHMCollateralLength = len(stablecoinsBorrowedWithGOHMCollateral)
path =file+'/stablecoinsBorrowedWithGOHMCollateralDeposits.csv'
stablecoinsBorrowedWithGOHMCollateral.to_csv(path, index = False)

path =file+'/stablecoinsBorrowedWithGOHMCollateralDeposits_piped.csv'
stablecoinsBorrowedWithGOHMCollateral.to_csv(path, index = False, sep='|')
print(path, 'file saved,', stablecoinsBorrowedWithGOHMCollateralLength,' records at', (dt.datetime.now()) )

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/stablecoinsBorrowedWithGOHMCollateralDeposits_piped.csv file saved, 27  records at 2023-02-21 11:40:28.157076


**********************************************************
MERGE EVERYTHING


In [138]:
stablecoinsBorrowedWithGOHMCollateral.head(1)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0x84778306b3f662f7d98e431591f302d924d143c3c0b2...,1677001823,2023-02-21 17:50:23,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,XAI,6540.0,silo,borrowStablecoin


In [139]:
stablecoinsBorrowedWithGOHMCollateralEvents = pd.DataFrame()
stablecoinsBorrowedWithGOHMCollateralEvents = pd.concat([gOHMCollateralDeposits,gOHMCollateralWithdrawals,stablecoinsBorrowedWithGOHMCollateral])
stablecoinsBorrowedWithGOHMCollateralEvents.head(1000)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0xcc1705094b3dd746ed1769d610d22bf12b7aec36e8a4...,1677001775,2023-02-21 17:49:35,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,gOHM,5.621351,silo,depositCollateral
1,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,1676673263,2023-02-17 22:34:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,gOHM,48.922199,silo,depositCollateral
2,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,1676555939,2023-02-16 13:58:59,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,gOHM,50.749992,silo,depositCollateral
3,0xe5163797f0922e4bded89126a213296a8c3d270a39e2...,1676383883,2023-02-14 14:11:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.180346,silo,depositCollateral
4,0xcc5e09e1e5c403b6925fa25f0410388de72ef1b45f30...,1676383523,2023-02-14 14:05:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.663108,silo,depositCollateral
...,...,...,...,...,...,...,...,...
22,0xbfc6df3a770233f4e0fbd6fe0cf8f6fa02f1432c570f...,1670906999,2022-12-13 04:49:59,0x339fd7c17ce670e7821f2f2ee94ddd49771483da,XAI,8000.000000,silo,borrowStablecoin
23,0x2485acd5a3853dc50921a27ea5bf8227871425cca3e4...,1670886647,2022-12-12 23:10:47,0x1ba67f30e25fc43863b50047d7fd4bd63b9c59ec,XAI,11900.000000,silo,borrowStablecoin
24,0x17fedebf20b120a55b895f96e2b466f2bbdcceb2b542...,1670881019,2022-12-12 21:36:59,0x3985caf03eef8f885edb1f8358143bdf08d8d48b,XAI,4000.000000,silo,borrowStablecoin
25,0x1aac54ba339ee99c2b3c69cbde905f3ddc7832ffd946...,1669246811,2022-11-23 23:40:11,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,XAI,262.000000,silo,borrowStablecoin


In [140]:
stablecoinsBorrowedWithGOHMCollateralEventsLength = len(stablecoinsBorrowedWithGOHMCollateralEvents)
path =file+'/stablecoinsBorrowedWithGOHMCollateralEvents.csv'
stablecoinsBorrowedWithGOHMCollateralEvents.to_csv(path, index = False)

path =file+'/stablecoinsBorrowedWithGOHMCollateralEvents_piped.csv'
stablecoinsBorrowedWithGOHMCollateralEvents.to_csv(path, index = False, sep='|')
print(path, 'file saved,', stablecoinsBorrowedWithGOHMCollateralEventsLength,' records at', (dt.datetime.now()) )

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/stablecoinsBorrowedWithGOHMCollateralEvents_piped.csv file saved, 74  records at 2023-02-21 11:40:48.108483


FRAX DAILY SNAPS

In [141]:
pairSnapshotsListLength=0
pairSnapshotsTicker = 0
Snapshots = pd.DataFrame()
pairSnapshotsDb = pd.DataFrame()
pairSnapshotsListLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit= False
while exit==False:
    #listLength=0
    skipValue = (pairSnapshotsTicker)*(1000)
    pairsQuery = frax.Query.pairPerDays(
         orderBy='id'
        ,orderDirection='desc'
        ,first=1000
        ,skipValue = skipValue
        #,where = [frax.Query.pairs.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
        ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52' }
        #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
    #{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

    )
    Snapshots= sg.query_df([
    pairsQuery.id
    ,pairsQuery.pair.collateral.symbol
    ,pairsQuery.pair.collateral.decimals
    ,pairsQuery.pair.asset.symbol
    ,pairsQuery.pair.asset.decimals
    ,pairsQuery.feeToProtocolRate
    ,pairsQuery.exchangeRate
    ,pairsQuery.totalAssetAmount
    ,pairsQuery.totalAssetShare
    ,pairsQuery.totalCollateral
    ,pairsQuery.totalBorrowAmount
    ,pairsQuery.totalBorrowShare
    ,pairsQuery.interestPerSecond
    ,pairsQuery.utilization
    ,pairsQuery.totalFeesAmount
    ,pairsQuery.totalFeesShare
    ,pairsQuery.lastAccrued
    ,pairsQuery.block
    ,pairsQuery.timestamp
    ,pairsQuery.totalAssetValue
    ,pairsQuery.totalCollateralValue
    ,pairsQuery.totalBorrowValue
    ,pairsQuery.pair
    ]
    )

    Snapshots.sort_values(by=['pairPerDays_timestamp'])

    pairSnapshotsDb=pd.concat([pairSnapshotsDb, Snapshots])
    recordTimestamp1 = pairSnapshotsDb.iat[pairSnapshotsTicker,14]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    pairSnapshotsDbLength = len(pairSnapshotsDb)
    pairSnapshotsListLength = len(Snapshots)
    recordID = pairSnapshotsDb.iat[pairSnapshotsTicker,0]
    Snapshot = pairSnapshotsDb.iat[pairSnapshotsTicker,1]
    print("iterations: ", pairSnapshotsTicker, "Lines skipped: ",skipValue, "records collected: ", pairSnapshotsListLength, " - latest recordID: ",Snapshot, " ", recordID , " - pairSnapshots DB length: ", pairSnapshotsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", pairSnapshotsTicker, "records collected: ", pairSnapshotsListLength, " - userdb length: ", pairSnapshotsDbLength)
    if daysAgo>daysLimit: exit =True
    if pairSnapshotsListLength<1000: exit =True
    pairSnapshotsTicker = pairSnapshotsTicker+1

print("Done. Total Records collected: ", pairSnapshotsDbLength)

iterations:  0 Lines skipped:  0 records collected:  85  - latest recordID:  gOHM   0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent  - pairSnapshots DB length:  85  - latest record from:  1969-12-31 16:00:00 19410 days ago
Done. Total Records collected:  85


In [142]:
pairSnapshotsDb.head(10)

,pairPerDays_id,pairPerDays_pair_collateral_symbol,pairPerDays_pair_collateral_decimals,pairPerDays_pair_asset_symbol,pairPerDays_pair_asset_decimals,pairPerDays_feeToProtocolRate,pairPerDays_exchangeRate,pairPerDays_totalAssetAmount,pairPerDays_totalAssetShare,pairPerDays_totalCollateral,...,pairPerDays_pair_collateral_id,pairPerDays_pair_oracleMultiplyAddress_id,pairPerDays_pair_oracleDivideAddress_id,pairPerDays_pair_rateContract_id,pairPerDays_pair_maxLTV,pairPerDays_pair_liquidationFee,pairPerDays_pair_maturity,pairPerDays_pair_pauseStatus,pairPerDays_pair_lenderWhitelistActive,pairPerDays_pair_borrowerWhitelistActive
0,0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent,gOHM,18,FRAX,18,0,363684761151992,5037338571043218964754023,4966638858282894118426824,3070504646113236841935,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
1,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19409,gOHM,18,FRAX,18,0,363684761151992,5037338571043218964754023,4966638858282894118426824,3070504646113236841935,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
2,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19408,gOHM,18,FRAX,18,0,353233237482579,5036458602328813008963501,4966638858282894118426824,3052722166378828892146,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
3,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19407,gOHM,18,FRAX,18,0,354538364355376,5035386265636883114384206,4966638858282894118426824,3035712082883572555858,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
4,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19406,gOHM,18,FRAX,18,0,353906299486322,5034722075873021602597750,4967011772748759733227735,3015712082883572555858,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
5,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19405,gOHM,18,FRAX,18,0,350112376599804,4698486647811909309719977,4636381316961784483066425,2938561823965620072232,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
6,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19404,gOHM,18,FRAX,18,0,363785846961333,4737561509933966578859758,4675953474727265989294250,2926144943608541855178,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
7,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19403,gOHM,18,FRAX,18,0,346065938064754,4696443773732938768853377,4636381316961784483066425,2926505753609477928993,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
8,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19402,gOHM,18,FRAX,18,0,355065538289602,4675315360621293243183076,4616545966840300870160156,2962449892162877274570,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
9,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19401,gOHM,

In [143]:
pairSnapshotsDb['pairPerDays_totalCollateralReadable'] = pairSnapshotsDb['pairPerDays_totalCollateral']/(10**(pairSnapshotsDb['pairPerDays_pair_collateral_decimals']))
pairSnapshotsDb['pairPerDays_totalCollateralReadable']=pairSnapshotsDb['pairPerDays_totalCollateralReadable'].astype('float')

pairSnapshotsDb['pairPerDays_totalAssetAmountReadable'] = pairSnapshotsDb['pairPerDays_totalAssetAmount']/(10**(pairSnapshotsDb['pairPerDays_pair_asset_decimals']))
pairSnapshotsDb['pairPerDays_totalAssetAmountReadable']=pairSnapshotsDb['pairPerDays_totalAssetAmountReadable'].astype('float')

pairSnapshotsDb['pairPerDays_totalBorrowAmountReadable'] = pairSnapshotsDb['pairPerDays_totalBorrowAmount']/(10**(pairSnapshotsDb['pairPerDays_pair_asset_decimals']))
pairSnapshotsDb['pairPerDays_totalBorrowAmountReadable']=pairSnapshotsDb['pairPerDays_totalBorrowAmountReadable'].astype('float')


pairSnapshotsDb['dateTime'] =(pd.to_datetime(pairSnapshotsDb['pairPerDays_timestamp'],unit='s'))

pairSnapshotsDb.drop_duplicates(keep ="first",inplace = True)


pairSnapshotsDb.head(2000)

,pairPerDays_id,pairPerDays_pair_collateral_symbol,pairPerDays_pair_collateral_decimals,pairPerDays_pair_asset_symbol,pairPerDays_pair_asset_decimals,pairPerDays_feeToProtocolRate,pairPerDays_exchangeRate,pairPerDays_totalAssetAmount,pairPerDays_totalAssetShare,pairPerDays_totalCollateral,...,pairPerDays_pair_maxLTV,pairPerDays_pair_liquidationFee,pairPerDays_pair_maturity,pairPerDays_pair_pauseStatus,pairPerDays_pair_lenderWhitelistActive,pairPerDays_pair_borrowerWhitelistActive,pairPerDays_totalCollateralReadable,pairPerDays_totalAssetAmountReadable,pairPerDays_totalBorrowAmountReadable,dateTime
0,0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent,gOHM,18,FRAX,18,0,363684761151992,5037338571043218964754023,4966638858282894118426824,3070504646113236841935,...,75000,10000,0,False,False,False,3070.504646,5.037339e+06,4.234425e+06,2023-02-21 17:32:59
1,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19409,gOHM,18,FRAX,18,0,363684761151992,5037338571043218964754023,4966638858282894118426824,3070504646113236841935,...,75000,10000,0,False,False,False,3070.504646,5.037339e+06,4.234425e+06,2023-02-21 17:32:59
2,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19408,gOHM,18,FRAX,18,0,353233237482579,5036458602328813008963501,4966638858282894118426824,3052722166378828892146,...,75000,10000,0,False,False,False,3052.722166,5.036459e+06,4.212153e+06,2023-02-20 23:07:59
3,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19407,gOHM,18,FRAX,18,0,354538364355376,5035386265636883114384206,4966638858282894118426824,3035712082883572555858,...,75000,10000,0,False,False,False,3035.712083,5.035386e+06,4.191081e+06,2023-02-19 20:16:23
4,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19406,gOHM,18,FRAX,18,0,353906299486322,5034722075873021602597750,4967011772748759733227735,3015712082883572555858,...,75000,10000,0,False,False,False,3015.712083,5.034722e+06,4.129513e+06,2023-02-18 22:49:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19330,gOHM,18,FRAX,18,0,416982985435782,3000265799749067769553552,2999681610314728238726349,1702267776392927486576,...,75000,10000,0,False,False,False,1702.267776,3.000266e+06,1.642246e+06,2022-12-04 23:12:59
81,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19329,gOHM,18,FRAX,18,0,435238970363534,3000407417762830870845360,2999931610314728238726349,1460557767872656691963,...,75000,10000,0,False,False,False,1460.557768,3.000407e+06,1.358687e+06,2022-12-03 23:30:59
82,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19328,gOHM,18,FRAX,18,0,424429606966213,3000316417810236657492429,2999931610314728238726349,1298896246642712535146,...,75000,10000,0,False,False,False,1298.896247,3.000316e+06,1.287539e+06,2022-12-02 23:55:59
83,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19327,gOHM,18,FRAX,18,0,455232217741157,1000250018374254924219488,1000177511746822029696096,116544304370705257,...,75000,10000,0,False,False,False,0.116544,1.000250e+06,1.821276e+02,2022-12-01 21:30:47


In [144]:
fraxPairSnapshotsLength = len(pairSnapshotsDb)
path =file+'/fraxPairSnapshotsLength.csv'
pairSnapshotsDb.to_csv(path, index = False)

path =file+'/fraxPairSnapshotsLength_piped.csv'
pairSnapshotsDb.to_csv(path, index = False, sep='|')
print(path, 'file saved,', fraxPairSnapshotsLength,' records at', (dt.datetime.now()) )

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/fraxPairSnapshotsLength_piped.csv file saved, 85  records at 2023-02-21 11:40:48.822122


In [145]:
fraxCollateral = pairSnapshotsDb.iat[pairSnapshotsTicker,37]
fraxCollateral

3070.504646113237

In [146]:
siloCollateralBalance

187.92484332506004

In [147]:
totalBorrowedUSDFrax= pairSnapshotsDb.iat[pairSnapshotsTicker,40]
totalBorrowedUSDFrax

Timestamp('2023-02-21 17:32:59')

In [148]:
collateralGohmLevels = pd.DataFrame({
    'protocol':['Frax','vesta','silo'],
    'collateralBalance':[fraxCollateral,0,siloCollateralBalance],
    'totalBorrowed':[totalBorrowedUSDFrax,0,totalBorrowedUSDSilo]
})
collateralGohmLevels.head(10)

,protocol,collateralBalance,totalBorrowed
0,Frax,3070.504646,2023-02-21 17:32:59
1,vesta,0.000000,0
2,silo,187.924843,571167.72762


In [149]:
collateralGohmLevelsLength = len(collateralGohmLevels)
path =file+'/collateralGohmLevels.csv'
collateralGohmLevels.to_csv(path, index = False)

path =file+'/collateralGohmLevels_piped.csv'
collateralGohmLevels.to_csv(path, index = False, sep='|')
print(path, 'file saved,', collateralGohmLevelsLength,' records at', (dt.datetime.now()) )

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/gohmAsStablecoinCollateral/data/collateralGohmLevels_piped.csv file saved, 3  records at 2023-02-21 11:40:48.875686


In [150]:
totalBorrowedUSDSilo

571167.7276195246